In [ ]:
from pint_pal.lite_utils import *
from pint_pal.noise_utils import *
from pint_pal.par_checker import *
from pint_pal.utils import *
from pint_pal.dmx_utils import *
from pint_pal.timingconfiguration import TimingConfiguration
from pint_pal.outlier_utils import *
from pint_pal.utils import apply_cut_flag, apply_cut_select
from pint_pal.plot_utils import plot_residuals_time
import pint_pal.report  # For reporting functionality!
import yaml
from astropy import log
import pint.fitter
from pint.utils import dmxparse
import os

# Uncomment to turn on interactive plotting:
#%matplotlib notebook

# For manual inspection:
from ipywidgets import *
from IPython.display import Image, display

In [ ]:
import glob

# Define the list of configs wanted to include in the report
configs_to_check = glob.glob("configs/*[0-9].nb.yaml")

In [ ]:
def append_to_reasonsDict(bad_entry, reasonsDict):
    """
    Assumes bad_entry has a reason, then determines whether to add
    to an existing reasonsDict key, or make a new one. Outputs the
    reasonsDict, which has reasons as keys, and lists of entries as
    values.
    """
    r = bad_entry[-1]  # reason
    if r in reasonsDict.keys():
        # Add to existing key
        reasonsDict[r].append(bad_entry)
    else:
        # Make a new key
        reasonsDict[r] = [bad_entry]
        
    return reasonsDict

# used to provide context for reasons employed
reason_infoDict = {
    'rfi':'used when individual channels appear to be affected by RFI not removed by nanopipe, \
    yet exceeding the snr threshold',
    'snr':'used for bad TOAs that barely exceed the snr threshold and otherwise look like outliers',
    'few_chans':'used when a `.ff` file has only one, or several, channels remaining post-nanopipe, \
    suggesting almost the entire band was affected by RFI',
    'non-detection':'used for cases that might otherwise be labeled `snr`, but the GTpd plot shows \
    a _very_ weak or non-existent signal',
    'unknown':'obvious outlier, but no apparent reason for badness in looking at the data cube',
    'smeared':'used in cases where there is an obvious slope in subints or general (unexplained), \
    but significant, pulse broadening',
    'isolated':'used when one/several (but very few) files-worth of TOAs are separated from the pack \
    by a long timespan',
}

In [ ]:
report = pint_pal.report.Report(
    title=f"TOA Excision Report", 
    sections=["Summary","Outlier Analysis","Reasons","Warnings"])
    # maybe it would be useful to have "procedure" to show order of ops?
report.begin_capturing_log("Warnings")

In [ ]:
report.add_markdown("Reasons",
                    f"""For the {len(configs_to_check)} config files included in this report, the following 
                    breakdown of reasons were employed to explain TOAs removed manually, after automated 
                    processes, using `bad-toa` and `bad-file` fields:
                    """
                   )

# Maybe use an astropy table instead?
#from astropy.table import QTable, Table, Column
#info_table = Table()

# Loop over config files and populate relevant dictionaries
no_manual_excision = []
yes_manual_excision = []
reasonsDict = {} # keys are reasons, values are corresponding bad-toa/file entries
epochdropDict = {}
btbfDict = {}
oaDict = {} # keys are sources, values are lists [n_outlier10, n_epochdrop]
for ctc in configs_to_check:
    tc = TimingConfiguration(ctc)
    bad_toas = tc.get_bad_toas()
    bad_files = tc.get_bad_files()
    if bad_toas:
        for bt in bad_toas:
            if len(bt) == 4: reasonsDict = append_to_reasonsDict(bt,reasonsDict)
            else: log.warning(f"Reason missing for bad-toa in {ctc}")
    if bad_files:
        for bf in bad_files:
            if len(bf) == 2:  reasonsDict = append_to_reasonsDict(bf,reasonsDict)
            else: log.warning(f"Reason missing for bad-file in {ctc}")
    if (not bad_toas) & (not bad_files):
        no_manual_excision.append(ctc)
    else:
        yes_manual_excision.append(ctc)
        
        if bad_toas: nbt = len(bad_toas)
        else: nbt = 0
        if bad_files: nbf = len(bad_files)
        else: nbf = 0
            
        btbfDict[os.path.basename(ctc)] = [nbt,nbf]
        
    # populate epochdrop dict (file/epochdrop value)
    if tc.get_excised():
        epochdrop_path = os.path.join(os.path.dirname(tc.get_excised()),'epochdrop.txt')
        
        f = open(epochdrop_path, "r")
        epochdrop_data = f.readlines()
        f.close()
        
        for line in epochdrop_data:
            datafile = line.strip().split()[0]
            ftest = line.strip().split()[4]      
            epochdropDict[datafile] = float(ftest)
            
        etf = open(tc.get_excised(), "r")
        timfile = etf.readlines()
        etf.close()
        
        nout,ned,ntoa,ncut = 0,0,0,0
        for line in timfile:
            if 'outlier10' in line: nout += 1
            if 'epochdrop' in line: ned += 1
            if 'NANOGrav' in line: ntoa += 1
            if '-cut' in line: ncut += 1
            
        oaDict[tc.get_source()] = [nout,ned,ntoa,ncut]
        
        
sources = np.array(list(oaDict.keys()))
        
# Probably a more concise way of grabbing these pieces of info
ncut_all = [oaDict[p][3] for p in oaDict.keys()]
ntoa_all = [oaDict[p][2] for p in oaDict.keys()]
noa_all = [oaDict[p][0] for p in oaDict.keys()]
ned_all = [oaDict[p][1] for p in oaDict.keys()]
out_all = [o+e for o,e in zip(noa_all,ned_all)]

tot_toas = np.sum(ntoa_all)
tot_cuts = np.sum(ncut_all)
tot_good = tot_toas - tot_cuts
tot_out = np.sum(out_all)

pct_cut = [float(c)/t for c,t in zip(ncut_all,ntoa_all)]
pct_out = [float(o)/t for o,t in zip(out_all,ntoa_all)]

In [ ]:
badfiles = []
totbadfile,totbadtoa = 0,0
for r in reasonsDict.keys():
    nbadfile,nbadtoa = 0,0
    for r_i in reasonsDict[r]:
        if len(r_i) == 2:
            nbadfile+=1
            badfiles.append(r_i[0])
        elif len(r_i) == 4:
            nbadtoa+=1
        else:
            log.warning(f"Something is wrong with manually excised data from {r_i[0]}")
            
    report.add_markdown("Reasons",
                    f"""- **{r}**: {len(reasonsDict[r])} total ({nbadtoa} `bad-toa`, {nbadfile} `bad-file`); \
                    {reason_infoDict[r]}.
                    """
                   )
    
    totbadfile += nbadfile
    totbadtoa += nbadtoa
    
report.add_markdown("Reasons",
                    f"""In these config files, there are {totbadtoa} `bad-toa` entries and {totbadfile}
                    `bad-file` entries. There are {len(no_manual_excision)} config files with no manual
                    excision entries.
                    """
                   )

In [ ]:
report.add_markdown("Outlier Analysis",
                    f"""Outlier analysis encompasses automatic TOA excision based on gibbs/epochdrop analyses;
                    the gibbs analysis determines the probability of individual TOAs being outliers (pout > 0.1
                    cut, `outlier10`) and the epochdrop analysis computes F-tests, dropping individual files
                    (F-test < 10<sup>-6</sup> cut, `epochdrop`). 
                    """
                   )

out_all = np.array(out_all)
pct_out = np.array(pct_out)
inds_2pct = np.where(pct_out > 0.02)[0]

report.add_markdown("Outlier Analysis",
                    f"""There are **{tot_out} TOAs cut by gibbs/epochdrop analyses** ({np.sum(noa_all)} `outlier10`, 
                    {np.sum(ned_all)} `epochdrop`) and {len(inds_2pct)} have > 2\% of their TOAs excised in this 
                    way. The following plots display remaining TOAs in gray, and highlight those removed by 
                    `outlier10` and `epochdrop` cuts in blue and orange, respectively. Two panels in each figure 
                    show the full range of residuals and a zoom-in, with limits defined by good TOAs in the latter 
                    case.
                    """
                   )

if len(inds_2pct):
    for i2p in inds_2pct:
        config = np.array(configs_to_check)[i2p]
        tc = TimingConfiguration(config)
        mo,to = tc.get_model_and_toas(excised=True)
        tc.manual_cuts(to)
        to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())
        highlight_cut_resids(to,mo,tc,cuts=['outlier10','epochdrop'],multi=True)
        
        s_noa = np.array(noa_all)[i2p]
        s_ned = np.array(ned_all)[i2p]
        report.add_plot("Outlier Analysis",
                        caption=f"""
                        Highlighted outliers for **{tc.get_source()}**; {out_all[i2p]} outliers flagged 
                        for this pulsar ({s_noa} `outlier10`, {s_ned} `epochdrop`; {pct_out[i2p]*100.0:.1f}\% 
                        of TOAs). Flagged outliers account for {pct_cut[i2p]*100.0:.1f}\% of all non-manual cuts.
                        """
                       )

In [ ]:
report.add_markdown("Summary",
                    f"""For the **{len(configs_to_check)} config files** included in this report, corresponding
                    excised TOA files contain {tot_toas} total TOAs; {tot_cuts} of those are cut via automated
                    routines/thresholds and an additional {totbadtoa}, plus TOAs from {totbadfile} bad-file
                    entries, are manually excised.
                    """
                   )

In [ ]:
report.generate_html(f"excision_report.html")
report.generate_pdf(f"excision_report.pdf")